# Build models

In [10]:
def build_model(n_gram, train_data_path):
    !mkdir -p "LMs"/"{n_gram}"
    !kenlm/build/bin/lmplz --text "{train_data_path}" --arpa "LMs"/"{n_gram}"/lm.arpa --order {n_gram} --discount_fallback  --verbose_header 
    !kenlm/build/bin/build_binary "LMs"/"{n_gram}"/lm.arpa "LMs"/"{n_gram}"/lm.binary
    
build_model(1, 'English-Mix/UNCorpus.train.tok')
# build_model(2, 'English-Mix/UNCorpus.train.tok')
# build_model(3, 'English-Mix/UNCorpus.train.tok')

=== 1/5 Counting and sorting n-grams ===
Reading fd 3
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2933681 types 17863
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:214356
Statistics:
1 17863 D1=0.532532 D2=1.06821 D3+=1.37723
Memory estimate for binary LM:
type     kB
probing 767 assuming -p 1.5
probing 837 assuming -r models -p 1.5
trie    519 without quantization
trie    467 assuming -q 8 -b 8 quantization 
trie    519 assuming -a 22 array pointer compression
trie    467 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:214356
=== 4/5 Calculating and writing order-interpolated probabilities ===
Chain sizes: 1:214356
=== 5/5 Writing ARPA model ===
----5---10---15---20---25---30---35---40---45---50---55---

# Test perplexity

In [5]:
def get_perplexity_and_OOVs(
    n_gram:int,
    test_data_path:str,
    models_path='LMs',
    overwrite_files=False,
    print_to_console=True,
):
    !mkdir -p "{models_path}"/"{n_gram}"/"{test_data_path}"

    # calculate and dump to a file
    !kenlm/build/bin/query "LMs"/"{test_data_path}"/"{n_gram}"/lm.binary < "{models_path}"/"{test_data_path}"/test_dataset.txt > "{models_path}"/"{test_data_path}"/results.txt

    with open(f"{models_path}/{test_data_path}/results.txt") as f:
        lines = f.read().splitlines()

    # collect
    perplexity_with_OOVs_line = lines[-4]
    perplexity_without_OOVs_line = lines[-3]
    counts_of_OOVs_line = lines[-2]

    perplexity_with_OOVs = float(perplexity_with_OOVs_line.split("Perplexity including OOVs:")[-1].strip())
    perplexity_without_OOVs = float(perplexity_without_OOVs_line.split("Perplexity excluding OOVs:")[-1].strip())
    counts_of_OOVs = int(counts_of_OOVs_line.split("OOVs:")[-1].strip())

    return perplexity_with_OOVs, perplexity_without_OOVs, counts_of_OOVs

print("1-gram")
print("UNCorpus.test.tok")
get_perplexity_and_OOVs(1, 'UNCorpus.test.tok')
print("UNCorpus.wiki.test.tok.tok")
get_perplexity_and_OOVs(1, 'UNCorpus.wiki.test.tok.tok')

print("2-gram")
print("UNCorpus.test.tok")
get_perplexity_and_OOVs(2, 'UNCorpus.test.tok')
print("UNCorpus.wiki.test.tok.tok")
get_perplexity_and_OOVs(2, 'UNCorpus.wiki.test.tok.tok')

print("3-gram")
print("UNCorpus.test.tok")
get_perplexity_and_OOVs(3, 'UNCorpus.test.tok')

1-gram
UNCorpus.test.tok
/bin/bash: LMs/UNCorpus.test.tok/test_dataset.txt: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'LMs/UNCorpus.test.tok/results.txt'